In [ ]:
# === Code ===
from pathlib import Path
import re
import pandas as pd
import plotly.express as px
from pathlib import Path
import re
import pandas as pd
import plotly.graph_objects as go

In [7]:
# --- config (edit these) ---
ROOT = "./glue_results_final"
TASK = "rte"
MODEL = "roberta-large"   # or "roberta-large"

# --- imports ---
from pathlib import Path
import re
import pandas as pd
import plotly.graph_objects as go

METHODS = ("full_finetuning", "lora", "rslora")

def parse_rank(run_name: str):
    m = re.search(r"_r(\d+)_", run_name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    # normalize column names
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy": "accuracy"})
    if "eval_loss" in df.columns and "loss" not in df.columns:
        df = df.rename(columns={"eval_loss": "loss"})
    # ensure epoch column
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index": "epoch"})
        df["epoch"] = df["epoch"] + 1
    # keep only epochs 1..5 if present
    df = df[df["epoch"].astype(float).between(1, 5)]
    return df

# --- load data (best runs only) ---
base = Path(ROOT) / TASK / MODEL
runs = []
for meth in METHODS:
    mdir = base / meth
    if not mdir.exists():
        continue
    for run in sorted([p for p in mdir.iterdir() if p.is_dir() and p.name.endswith("_best_accuracy")]):
        df = read_eval(run / "eval_per_epoch.csv")
        if df is None or "accuracy" not in df.columns:
            continue
        df = df.sort_values("epoch")
        runs.append({
            "method": meth,
            "rank": parse_rank(run.name) if meth != "full_finetuning" else None,
            "name": run.name,
            "df": df
        })

# group helpers
full = next((r for r in runs if r["method"] == "full_finetuning"), None)
ranks = sorted({r["rank"] for r in runs if r["rank"] is not None})

# --- build figure: Accuracy vs Epoch with rank dropdown ---
fig = go.Figure()

# trace 0: full_finetuning (static)
if full is not None:
    fig.add_trace(go.Scatter(
        x=full["df"]["epoch"], y=full["df"]["accuracy"],
        mode="lines+markers",
        name="full_finetuning",
        hovertext=[full["name"]]*len(full["df"]),
        visible=True
    ))
else:
    fig.add_trace(go.Scatter(x=[], y=[], name="full_finetuning", visible=True))

# pick initial rank (first available)
r0 = ranks[9] if ranks else None
def slice_rank(method, r):
    for item in runs:
        if item["method"] == method and item["rank"] == r:
            return item["df"], item["name"]
    return pd.DataFrame({"epoch": [], "accuracy": []}), ""

# trace 1: lora at r0
df_l0, nm_l0 = slice_rank("lora", r0) if r0 is not None else (pd.DataFrame({"epoch": [], "accuracy": []}), "")
fig.add_trace(go.Scatter(
    x=df_l0["epoch"], y=df_l0["accuracy"], mode="lines+markers",
    name=f"lora_r{r0}" if r0 is not None else "lora",
    hovertext=[nm_l0]*len(df_l0), visible=True
))

# trace 2: rslora at r0
df_r0, nm_r0 = slice_rank("rslora", r0) if r0 is not None else (pd.DataFrame({"epoch": [], "accuracy": []}), "")
fig.add_trace(go.Scatter(
    x=df_r0["epoch"], y=df_r0["accuracy"], mode="lines+markers",
    name=f"rslora_r{r0}" if r0 is not None else "rslora",
    hovertext=[nm_r0]*len(df_r0), visible=True
))

# dropdown to switch rank
buttons = []
for r in ranks:
    dfl, nml = slice_rank("lora", r)
    dfr, nmr = slice_rank("rslora", r)

    # traces: 0=full_ft, 1=lora, 2=rslora
    buttons.append(dict(
        label=f"rank {r}",
        method="update",
        args=[
            {"x": [
                full["df"]["epoch"] if full else [],  # trace 0
                dfl["epoch"],                         # trace 1
                dfr["epoch"],                         # trace 2
            ],
             "y": [
                full["df"]["accuracy"] if full else [],
                dfl["accuracy"],
                dfr["accuracy"],
             ],
             "name": [
                "full_finetuning",
                f"lora_r{r}",
                f"rslora_r{r}",
             ],
             "hovertext": [
                [full["name"]]*len(full["df"]) if full else [],
                [nml]*len(dfl),
                [nmr]*len(dfr),
             ]},
            {"title": f"{TASK} · {MODEL} · Accuracy vs Epoch (rank {r})"}
        ]
    ))

fig.update_layout(
    title=f"{TASK} · {MODEL} · Accuracy vs Epoch" + (f" (rank {r0})" if r0 is not None else ""),
    xaxis_title="epoch",
    yaxis_title="accuracy",
    legend_title_text="method",
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        showactive=True,
        x=1.0, xanchor="right",
        y=1.1, yanchor="top"
    )] if ranks else []
)

fig


In [8]:
# === Config ===
ROOT = "./glue_results_final"   # parent folder holding the 'task' dir
TASK = "rte"                    # e.g., "rte", "mrpc", "cola"
MODEL = "roberta-base"          # "roberta-base" or "roberta-large"

# === Code ===
from pathlib import Path
import re
import pandas as pd
import plotly.graph_objects as go

METHODS = ("full_finetuning", "lora", "rslora")

def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    # normalize possible names
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy": "accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    # focus on epochs 1..5 if present
    df = df[df["epoch"].astype(float).between(1,5)]
    return df.sort_values("epoch")

def load_best_runs(root: str, task: str, model: str):
    base = Path(root) / task / model
    runs = []
    for method in METHODS:
        mdir = base / method
        if not mdir.exists():
            continue
        for run in sorted([p for p in mdir.iterdir() if p.is_dir() and p.name.endswith("_best_accuracy")]):
            df = read_eval(run / "eval_per_epoch.csv")
            if df is None or "accuracy" not in df.columns:
                continue
            runs.append({
                "method": method,
                "rank": parse_rank(run.name) if method != "full_finetuning" else None,
                "name": run.name,
                "df": df
            })
    return runs

def dash_for_rank(rank: int, ranks_sorted: list[int]) -> str:
    """
    Map rank position -> dash style (more solid for higher ranks).
    Uses an ordered palette: dot -> dashdot -> dash -> longdash -> solid.
    """
    palette = ["dot", "dashdot", "dash", "longdash", "solid"]
    if not ranks_sorted:
        return "solid"
    idx = ranks_sorted.index(rank)
    frac = idx / max(1, len(ranks_sorted) - 1)
    palette_idx = int(round(frac * (len(palette) - 1)))
    return palette[palette_idx]

# load
runs = load_best_runs(ROOT, TASK, MODEL)
if not runs:
    raise SystemExit(f"No *_best_accuracy runs found under {ROOT}/{TASK}/{MODEL}")

# collect rank list (for dash mapping)
all_ranks = sorted({r["rank"] for r in runs if r["rank"] is not None})

# colors
COLOR = {
    "full_finetuning": "black",
    "lora": "red",
    "rslora": "green"
}

fig = go.Figure()

# Add full FT first (usually just one best run)
for r in [x for x in runs if x["method"] == "full_finetuning"]:
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"full_finetuning",
        legendgroup="full_finetuning",
        line=dict(color=COLOR["full_finetuning"], dash="solid"),
        hovertext=[r["name"]]*len(r["df"])
    ))

# Add all LoRA ranks
for r in [x for x in runs if x["method"] == "lora"]:
    dash = dash_for_rank(r["rank"], all_ranks)
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"lora_r{r['rank']}",
        legendgroup="lora",
        line=dict(color=COLOR["lora"], dash=dash),
        hovertext=[r["name"]]*len(r["df"])
    ))

# Add all RSLoRA ranks
for r in [x for x in runs if x["method"] == "rslora"]:
    dash = dash_for_rank(r["rank"], all_ranks)
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"rslora_r{r['rank']}",
        legendgroup="rslora",
        line=dict(color=COLOR["rslora"], dash=dash),
        hovertext=[r["name"]]*len(r["df"])
    ))

fig.update_layout(
    title=f"{TASK} · {MODEL} · Accuracy vs Epoch (all best runs)",
    xaxis_title="epoch",
    yaxis_title="accuracy",
    legend_title_text="method / rank",
    hovermode="x unified"
)

fig


In [9]:
# === Config ===
ROOT = "./glue_results_final"   # parent folder holding the 'task' dir
TASK = "rte"                    # e.g., "rte", "mrpc", "cola"
MODEL = "roberta-base"          # "roberta-base" or "roberta-large"

# === Code ===
from pathlib import Path
import re
import pandas as pd
import plotly.graph_objects as go

METHODS = ("full_finetuning", "lora", "rslora")

def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    # normalize possible names
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy": "accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    # focus on epochs 1..5 if present
    df = df[df["epoch"].astype(float).between(1,5)]
    return df.sort_values("epoch")

def load_best_runs(root: str, task: str, model: str):
    base = Path(root) / task / model
    runs = []
    for method in METHODS:
        mdir = base / method
        if not mdir.exists():
            continue
        for run in sorted([p for p in mdir.iterdir() if p.is_dir() and p.name.endswith("_best_accuracy")]):
            df = read_eval(run / "eval_per_epoch.csv")
            if df is None or "accuracy" not in df.columns:
                continue
            runs.append({
                "method": method,
                "rank": parse_rank(run.name) if method != "full_finetuning" else None,
                "name": run.name,
                "df": df
            })
    return runs

def dash_for_rank(rank: int, ranks_sorted: list[int]) -> str:
    """
    Map rank position -> dash style (more solid for higher ranks).
    Uses an ordered palette: dot -> dashdot -> dash -> longdash -> solid.
    """
    palette = ["dot", "dashdot", "dash", "longdash", "solid"]
    if not ranks_sorted:
        return "solid"
    idx = ranks_sorted.index(rank)
    frac = idx / max(1, len(ranks_sorted) - 1)
    palette_idx = int(round(frac * (len(palette) - 1)))
    return palette[palette_idx]

# load
runs = load_best_runs(ROOT, TASK, MODEL)
if not runs:
    raise SystemExit(f"No *_best_accuracy runs found under {ROOT}/{TASK}/{MODEL}")

# collect rank list (for dash mapping)
all_ranks = sorted({r["rank"] for r in runs if r["rank"] is not None})

# colors
COLOR = {
    "full_finetuning": "black",
    "lora": "red",
    "rslora": "green"
}

fig = go.Figure()

# Add full FT first (usually just one best run)
for r in [x for x in runs if x["method"] == "full_finetuning"]:
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name="full_finetuning",
        line=dict(color=COLOR["full_finetuning"], dash="solid"),
        hovertext=[r["name"]]*len(r["df"])
    ))

# Add all LoRA ranks (each trace toggles individually)
for r in sorted([x for x in runs if x["method"] == "lora"], key=lambda z: z["rank"] or -1):
    dash = dash_for_rank(r["rank"], all_ranks)
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"lora_r{r['rank']}",
        line=dict(color=COLOR["lora"], dash=dash),
        hovertext=[r["name"]]*len(r["df"])
    ))

# Add all RSLoRA ranks (each trace toggles individually)
for r in sorted([x for x in runs if x["method"] == "rslora"], key=lambda z: z["rank"] or -1):
    dash = dash_for_rank(r["rank"], all_ranks)
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"rslora_r{r['rank']}",
        line=dict(color=COLOR["rslora"], dash=dash),
        hovertext=[r["name"]]*len(r["df"])
    ))

fig.update_layout(
    title=f"{TASK} · {MODEL} · Accuracy vs Epoch (all best runs)",
    xaxis_title="epoch",
    yaxis_title="accuracy",
    legend_title_text="method / rank",
    hovermode="x unified"
)

fig


In [12]:
# === Config ===
ROOT = "./glue_results_final"
TASK = "rte"
MODEL = "roberta-large"  # or "roberta-large"

# === Code ===
from pathlib import Path
import re
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import sample_colorscale

METHODS = ("full_finetuning", "lora", "rslora")

def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path) -> pd.DataFrame | None:
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy": "accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    df = df[df["epoch"].astype(float).between(1,5)]
    return df.sort_values("epoch")

def load_best_runs(root: str, task: str, model: str):
    base = Path(root) / task / model
    runs = []
    for method in METHODS:
        mdir = base / method
        if not mdir.exists():
            continue
        for run in sorted([p for p in mdir.iterdir() if p.is_dir() and p.name.endswith("_best_accuracy")]):
            df = read_eval(run / "eval_per_epoch.csv")
            if df is None or "accuracy" not in df.columns:
                continue
            runs.append({
                "method": method,
                "rank": parse_rank(run.name) if method != "full_finetuning" else None,
                "name": run.name,
                "df": df
            })
    return runs

def make_rank_colors(ranks_sorted, colorscale_name):
    """
    Map each rank -> a distinct shade sampled from a colorscale.
    We sample from 0.15..0.95 to avoid too-light/too-dark extremes.
    """
    if not ranks_sorted:
        return {}
    n = len(ranks_sorted)
    colors = {}
    for i, r in enumerate(ranks_sorted):
        t = 0.15 + (0.80 * (i / (n - 1 if n > 1 else 1)))
        colors[r] = sample_colorscale(colorscale_name, [t])[0]  # returns a hex string
    return colors

# Load data
runs = load_best_runs(ROOT, TASK, MODEL)
if not runs:
    raise SystemExit(f"No *_best_accuracy runs found under {ROOT}/{TASK}/{MODEL}")

# Rank set for color mapping
all_ranks = sorted({r["rank"] for r in runs if r["rank"] is not None})

# Color maps
FULL_COLOR = "black"
lora_colors   = make_rank_colors(all_ranks, "Reds")
rslora_colors = make_rank_colors(all_ranks, "Greens")

# Optional: small linewidth ramp with rank (helps distinguish similar shades)
def line_width_for_rank(rank, ranks_sorted):
    if not ranks_sorted or rank is None:
        return 2
    idx = ranks_sorted.index(rank)
    base, bonus = 2.0, 1.0  # base width + up to +1
    return base + bonus * (idx / (len(ranks_sorted) - 1 if len(ranks_sorted) > 1 else 1))

fig = go.Figure()

# Full FT (usually single run)
for r in [x for x in runs if x["method"] == "full_finetuning"]:
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name="full_finetuning",
        line=dict(color=FULL_COLOR, dash="solid", width=2.5),
        hovertext=[r["name"]]*len(r["df"])
    ))

# LoRA (each rank gets its own red shade)
for r in sorted([x for x in runs if x["method"] == "lora"], key=lambda z: z["rank"] or -1):
    rk = r["rank"]
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"lora_r{rk}",
        line=dict(color=lora_colors.get(rk, "red"), dash="solid",
                  width=line_width_for_rank(rk, all_ranks)),
        hovertext=[r["name"]]*len(r["df"])
    ))

# RSLoRA (each rank gets its own green shade)
for r in sorted([x for x in runs if x["method"] == "rslora"], key=lambda z: z["rank"] or -1):
    rk = r["rank"]
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"rslora_r{rk}",
        line=dict(color=rslora_colors.get(rk, "green"), dash="solid",
                  width=line_width_for_rank(rk, all_ranks)),
        hovertext=[r["name"]]*len(r["df"])
    ))

fig.update_layout(
    title=f"{TASK} · {MODEL} · Accuracy vs Epoch (all best runs)",
    xaxis_title="epoch",
    yaxis_title="accuracy",
    legend_title_text="method / rank",
    hovermode="x unified"
)

fig


In [13]:








# Marker symbols to assign per rank (wraps if > len)
MARKERS = [
    "circle","square","diamond","cross","x",
    "triangle-up","triangle-down","triangle-left","triangle-right","star"
]

def marker_for_rank(rank, ranks_sorted):
    if not ranks_sorted or rank is None:
        return "circle"
    i = ranks_sorted.index(rank)
    return MARKERS[i % len(MARKERS)]

# === Build Figure ===
fig = go.Figure()

# Full FT (usually single run)
for r in [x for x in runs if x["method"] == "full_finetuning"]:
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name="full_finetuning",
        line=dict(color=FULL_COLOR, dash="solid", width=2.5),
        marker=dict(symbol="circle", size=8, color=FULL_COLOR),
        hovertext=[r["name"]]*len(r["df"])
    ))

# LoRA (each rank gets its own red shade + marker)
for r in sorted([x for x in runs if x["method"] == "lora"], key=lambda z: z["rank"] or -1):
    rk = r["rank"]
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"lora_r{rk}",
        line=dict(color=lora_colors.get(rk, "red"), width=line_width_for_rank(rk, all_ranks)),
        marker=dict(symbol=marker_for_rank(rk, all_ranks), size=8,
                    color=lora_colors.get(rk, "red")),
        hovertext=[r["name"]]*len(r["df"])
    ))

# RSLoRA (each rank gets its own green shade + marker)
for r in sorted([x for x in runs if x["method"] == "rslora"], key=lambda z: z["rank"] or -1):
    rk = r["rank"]
    fig.add_trace(go.Scatter(
        x=r["df"]["epoch"], y=r["df"]["accuracy"],
        mode="lines+markers",
        name=f"rslora_r{rk}",
        line=dict(color=rslora_colors.get(rk, "green"), width=line_width_for_rank(rk, all_ranks)),
        marker=dict(symbol=marker_for_rank(rk, all_ranks), size=8,
                    color=rslora_colors.get(rk, "green")),
        hovertext=[r["name"]]*len(r["df"])
    ))

fig.update_layout(
    title=f"{TASK} · {MODEL} · Accuracy vs Epoch (all best runs)",
    xaxis_title="epoch",
    yaxis_title="accuracy",
    legend_title_text="method / rank",
    hovermode="x unified"
)

fig


In [55]:
df

,method,rank,run_name,accuracy
0,full_finetuning,None,full_finetuning_lr3.1622776601683795e-05_best_...,0.794224


In [56]:
# === Config ===
ROOT = "./glue_results_final"
TASK = "rte"
MODEL = "roberta-base"   # or "roberta-base"
METHOD_TO_SHOW = "rslora"   # or "rslora"

def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path):
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy":"accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    df = df[df["epoch"].astype(float).between(1,5)].sort_values("epoch")
    return df

def epoch5_accuracy(df):
    row = df[df["epoch"].astype(float) == 5.0]
    if not row.empty:
        return float(row["accuracy"].iloc[-1])
    return float(df["accuracy"].iloc[-1]) if not df.empty else None

def load_points(root, task, model, method):
    base = Path(root)/task/model
    rows = []

    # full fine-tuning
    mdir = base/"full_finetuning"
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                df = read_eval(run/"eval_per_epoch.csv")
                if df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":"full_finetuning","rank":None,"run_name":run.name,"accuracy":acc})

    # chosen method (lora or rslora)
    mdir = base/method
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                rk = parse_rank(run.name)
                df = read_eval(run/"eval_per_epoch.csv")
                if rk is not None and df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":method,"rank":rk,"run_name":run.name,"accuracy":acc})
    return pd.DataFrame(rows)

# Load
df = load_points(ROOT, TASK, MODEL, METHOD_TO_SHOW)
if df.empty:
    raise SystemExit("No runs found.")

# Separate LoRA and full_ft
lora_df = df[df["method"]==METHOD_TO_SHOW].sort_values("rank")
ft_acc = df.loc[df["method"]=="full_finetuning","accuracy"].mean()

# Plot
fig = px.scatter(
    lora_df, x="rank", y="accuracy",
    color="method", symbol="method",
    color_discrete_map={METHOD_TO_SHOW:"red"},
    # hover_data=["run_name"],
    title=f"{TASK} · {MODEL} · {METHOD_TO_SHOW.upper()} Rank vs Accuracy @ epoch 5",
    labels={"rank":"LoRA rank", "accuracy":"accuracy"}
)

# Add trend line for LoRA
fig.add_trace(go.Scatter(
    x=lora_df["rank"], y=lora_df["accuracy"],
    mode="lines", name=f"{METHOD_TO_SHOW} (trend)",
    line=dict(color="red", width=2),
    hoverinfo="skip",
    showlegend=False
))

# Add horizontal line for full fine-tuning
fig.add_trace(go.Scatter(
    x=[min(ranks), max(ranks)],
    y=[ft_acc, ft_acc],
    mode="lines",
    name="full_finetuning",   # legend label
    line=dict(color="black", dash="dot", width=2),
    hovertemplate="method=full_finetuning<br>accuracy=%{y:.3f}<extra></extra>",

))


fig.update_layout(
    xaxis_title="rank",
    yaxis_title="accuracy",
    legend_title_text="Method",
)

fig


In [57]:
lora_df

,method,rank,run_name,accuracy
4,rslora,1.0,rslora_r1_alpha1_lr0.001_best_accuracy,0.750903
2,rslora,4.0,rslora_r4_alpha4_lr0.001_best_accuracy,0.768953
10,rslora,8.0,rslora_r8_alpha8_lr0.00031622776601683794_best...,0.743682
9,rslora,16.0,rslora_r16_alpha16_lr0.0001_best_accuracy,0.722022
8,rslora,32.0,rslora_r32_alpha32_lr0.00031622776601683794_be...,0.527076
5,rslora,64.0,rslora_r64_alpha64_lr3.16227766e-05_best_accuracy,0.714801
3,rslora,128.0,rslora_r128_alpha128_lr1e-05_best_accuracy,0.718412
7,rslora,256.0,rslora_r256_alpha256_lr0.001_best_accuracy,0.527076
6,rslora,512.0,rslora_r512_alpha512_lr0.0001_best_accuracy,0.527076
1,rslora,768.0,rslora_r768_alpha768_lr0.00031622776601683794_...,0.527076


In [31]:
ft_acc = df.loc[df["method"]=="full_finetuning","accuracy"]

In [29]:
ft_acc

0    0.794224
Name: accuracy, dtype: float64

In [32]:
df

,method,rank,run_name,accuracy
0,full_finetuning,NaN,full_finetuning_lr3.1622776601683795e-05_best_...,0.794224
1,lora,1.0,lora_r1_alpha1_lr0.001_best_accuracy,0.750903
2,lora,16.0,lora_r16_alpha16_lr0.00031622776601683794_best...,0.718412
3,lora,8.0,lora_r8_alpha8_lr0.001_best_accuracy,0.765343
4,lora,32.0,lora_r32_alpha32_lr0.00031622776601683794_best...,0.779783
5,lora,128.0,lora_r128_alpha128_lr5.623413251903491e-05_bes...,0.722022
6,lora,4.0,lora_r4_alpha4_lr0.001_best_accuracy,0.754513
7,lora,64.0,lora_r64_alpha64_lr0.0001_best_accuracy,0.740072
8,lora,256.0,lora_r256_alpha256_lr3.1622776601683795e-05_be...,0.722022
9,lora,512.0,lora_r512_alpha512_lr3.1622776601683795e-05_be...,0.722022


In [61]:
# === Config ===
ROOT = "./glue_results_final"
TASK = "rte"
MODEL = "roberta-large"        # "roberta-base" or "roberta-large"
METHOD_TO_SHOW = "lora"     # "lora" or "rslora"


def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path):
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy":"accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    df = df[df["epoch"].astype(float).between(1,5)].sort_values("epoch")
    return df

def epoch5_accuracy(df):
    row = df[df["epoch"].astype(float) == 5.0]
    if not row.empty:
        return float(row["accuracy"].iloc[-1])
    return float(df["accuracy"].iloc[-1]) if not df.empty else None

def load_points(root, task, model, method):
    base = Path(root)/task/model
    rows = []

    # full fine-tuning
    mdir = base/"full_finetuning"
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                df = read_eval(run/"eval_per_epoch.csv")
                if df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":"full_finetuning","rank":None,"run_name":run.name,"accuracy":acc})

    # chosen method (lora or rslora)
    mdir = base/method
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                rk = parse_rank(run.name)
                df = read_eval(run/"eval_per_epoch.csv")
                if rk is not None and df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":method,"rank":rk,"run_name":run.name,"accuracy":acc})
    return pd.DataFrame(rows)

# ---- build figure (dynamic axes) ----
df = load_points(ROOT, TASK, MODEL, METHOD_TO_SHOW)
if df.empty:
    raise SystemExit("No runs found.")

meth_df = df[df["method"]==METHOD_TO_SHOW].dropna(subset=["rank"]).sort_values("rank")
if meth_df.empty:
    raise SystemExit(f"No {METHOD_TO_SHOW} ranks found.")

# dynamic ranks list for THIS model/method
ranks = meth_df["rank"].astype(int).tolist()
xmin, xmax = min(ranks), max(ranks)

# full-FT baseline
ft_acc = df.loc[df["method"]=="full_finetuning","accuracy"].mean()

# color per method
method_color = {"lora":"red", "rslora":"green"}
line_color = method_color[METHOD_TO_SHOW]
title_method = "LoRA" if METHOD_TO_SHOW=="lora" else "RSLoRA"

# Separate LoRA and full_ft
lora_df = df[df["method"]==METHOD_TO_SHOW].sort_values("rank")
ft_acc = df.loc[df["method"]=="full_finetuning","accuracy"].mean()

# Plot
fig = px.scatter(
    lora_df, x="rank", y="accuracy",
    color="method", symbol="method",
    color_discrete_map={METHOD_TO_SHOW:"red"},
    # hover_data=["run_name"],
    title=f"{TASK} · {MODEL} · {METHOD_TO_SHOW.upper()} Rank vs Accuracy @ epoch 5",
    labels={"rank":"LoRA rank", "accuracy":"accuracy"}
)

# Add trend line for LoRA
fig.add_trace(go.Scatter(
    x=lora_df["rank"], y=lora_df["accuracy"],
    mode="lines", name=f"{METHOD_TO_SHOW} (trend)",
    line=dict(color="red", width=2),
    hoverinfo="skip",
    showlegend=False
))

# Add horizontal line for full fine-tuning
fig.add_trace(go.Scatter(
    x=[min(ranks), max(ranks)],
    y=[ft_acc, ft_acc],
    mode="lines",
    name="full_finetuning",   # legend label
    line=dict(color="black", dash="dot", width=2),
    hovertemplate="method=full_finetuning<br>accuracy=%{y:.3f}<extra></extra>",

))


fig.update_layout(
    xaxis_title="rank",
    yaxis_title="accuracy",
    legend_title_text="Method",
)

fig


In [70]:
# === Config ===
ROOT = "./glue_results_final"
TASK = "rte"
MODEL = "roberta-large"        # "roberta-base" or "roberta-large"
METHOD_TO_SHOW = "lora"     # "lora" or "rslora"

# === Code ===
from pathlib import Path
import re
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def parse_rank(name: str):
    m = re.search(r"_r(\d+)_", name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path):
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None
    if "eval_accuracy" in df.columns and "accuracy" not in df.columns:
        df = df.rename(columns={"eval_accuracy":"accuracy"})
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index":"epoch"})
        df["epoch"] = df["epoch"] + 1
    df = df[df["epoch"].astype(float).between(1,5)].sort_values("epoch")
    return df

def epoch5_accuracy(df):
    row = df[df["epoch"].astype(float) == 5.0]
    if not row.empty:
        return float(row["accuracy"].iloc[-1])
    return float(df["accuracy"].iloc[-1]) if not df.empty else None

def load_points(root, task, model, method):
    base = Path(root)/task/model
    rows = []

    # full fine-tuning
    mdir = base/"full_finetuning"
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                df = read_eval(run/"eval_per_epoch.csv")
                if df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":"full_finetuning","rank":None,"run_name":run.name,"accuracy":acc})

    # chosen method (lora or rslora)
    mdir = base/method
    if mdir.exists():
        for run in mdir.iterdir():
            if run.is_dir() and run.name.endswith("_best_accuracy"):
                rk = parse_rank(run.name)
                df = read_eval(run/"eval_per_epoch.csv")
                if rk is not None and df is not None and "accuracy" in df.columns:
                    acc = epoch5_accuracy(df)
                    if acc is not None:
                        rows.append({"method":method,"rank":rk,"run_name":run.name,"accuracy":acc})
    return pd.DataFrame(rows)

# ---- build figure (dynamic axes) ----
df = load_points(ROOT, TASK, MODEL, METHOD_TO_SHOW)
if df.empty:
    raise SystemExit("No runs found.")

meth_df = df[df["method"]==METHOD_TO_SHOW].dropna(subset=["rank"]).sort_values("rank")
if meth_df.empty:
    raise SystemExit(f"No {METHOD_TO_SHOW} ranks found.")

# dynamic ranks list for THIS model/method
ranks = meth_df["rank"].astype(int).tolist()
xmin, xmax = min(ranks), max(ranks)

# full-FT baseline
ft_acc = df.loc[df["method"]=="full_finetuning","accuracy"].mean()

# color per method
method_color = {"lora":"red", "rslora":"green"}
line_color = method_color[METHOD_TO_SHOW]
title_method = "LoRA" if METHOD_TO_SHOW=="lora" else "RSLoRA"

# points
fig = px.scatter(
    meth_df, x="rank", y="accuracy",
    color="method", symbol="method",
    color_discrete_map={METHOD_TO_SHOW: line_color},
    hover_data=[],  # keep hover clean
    title=f"{TASK} · {MODEL} · {title_method} Rank vs Accuracy @ Epoch 5",
    labels={"rank":f"{title_method} rank", "accuracy":"accuracy"}
)

# trend line (no legend)
fig.add_trace(go.Scatter(
    x=meth_df["rank"], y=meth_df["accuracy"],
    mode="lines",
    line=dict(color=line_color, width=2),
    hoverinfo="skip",
    showlegend=False
))

# full-FT as proper legend trace
if pd.notna(ft_acc):
    fig.add_trace(go.Scatter(
        x=[xmin, xmax],
        y=[ft_acc, ft_acc],
        mode="lines",
        name="full_finetuning",
        line=dict(color="black", dash="dot", width=2),
        hovertemplate="method=full_finetuning<br>accuracy=%{y:.3f}<extra></extra>",
    ))


fig.update_layout(
    xaxis_title=f"{title_method} rank",
    yaxis_title="accuracy",
    legend_title_text="Method",
    margin=dict(l=60, r=20, t=60, b=60),
)

fig


In [71]:
df

,method,rank,run_name,accuracy
0,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,0.819495
1,lora,1.0,lora_r1_alpha1_lr0.001_best_accuracy,0.830325
2,lora,16.0,lora_r16_alpha16_lr0.00031622776601683794_best...,0.848375
3,lora,32.0,lora_r32_alpha32_lr0.0001_best_accuracy,0.819495
4,lora,8.0,lora_r8_alpha8_lr0.00031622776601683794_best_a...,0.819495
5,lora,1024.0,lora_r1024_alpha1024_lr1.77827941e-05_best_acc...,0.711191
6,lora,128.0,lora_r128_alpha128_lr5.623413251903491e-05_bes...,0.841155
7,lora,4.0,lora_r4_alpha4_lr0.001_best_accuracy,0.855596
8,lora,64.0,lora_r64_alpha64_lr0.00031622776601683794_best...,0.851986
9,lora,256.0,lora_r256_alpha256_lr3.1622776601683795e-05_be...,0.844765


In [ ]:
# === Config ===
ROOT  = "./glue_results_final"
TASK  = "rte"                 # e.g., "rte", "mrpc", "cola"
MODEL = "roberta-large"       # "roberta-base" or "roberta-large"

# === Code ===


METHODS = ("full_finetuning", "lora", "rslora")

def parse_rank(run_name: str) -> int | None:
    m = re.search(r"_r(\d+)_", run_name)
    return int(m.group(1)) if m else None

def read_eval(csv_path: Path) -> pd.DataFrame | None:
    """Load eval_per_epoch.csv and normalize columns to eval_* names."""
    if not csv_path.exists():
        return None
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None

    # Normalize common column names
    ren = {}
    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index": "epoch"})
        df["epoch"] = df["epoch"] + 1

    # map eval_loss / loss -> eval_loss
    if "eval_loss" in df.columns:
        ren["eval_loss"] = "eval_loss"
    elif "loss" in df.columns:
        ren["loss"] = "eval_loss"

    # map eval_accuracy / accuracy -> eval_accuracy
    if "eval_accuracy" in df.columns:
        ren["eval_accuracy"] = "eval_accuracy"
    elif "accuracy" in df.columns:
        ren["accuracy"] = "eval_accuracy"

    # optional speed/runtime fields
    if "runtime" in df.columns:               ren["runtime"] = "eval_runtime"
    if "samples_per_second" in df.columns:    ren["samples_per_second"] = "eval_samples_per_second"
    if "steps_per_second" in df.columns:      ren["steps_per_second"] = "eval_steps_per_second"

    if ren:
        df = df.rename(columns=ren)

    # keep only relevant columns + epoch
    keep = [c for c in df.columns if c.startswith("eval_")] + ["epoch"]
    df = df[keep].copy()
    return df.sort_values("epoch")

def read_train(csv_path: Path) -> pd.DataFrame | None:
    """Load train_per_epoch.csv and normalize columns to train_* names."""
    if not csv_path.exists():
        return None
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None

    if "epoch" not in df.columns:
        df = df.reset_index().rename(columns={"index": "epoch"})
        df["epoch"] = df["epoch"] + 1

    # map train_loss / loss -> train_loss
    ren = {}
    if "train_loss" in df.columns:
        ren["train_loss"] = "train_loss"
    elif "loss" in df.columns:
        ren["loss"] = "train_loss"

    # optional train_* speed/runtime fields (if present)
    for raw, std in [
        ("runtime", "train_runtime"),
        ("samples_per_second", "train_samples_per_second"),
        ("steps_per_second", "train_steps_per_second"),
        ("train_runtime", "train_runtime"),
        ("train_samples_per_second", "train_samples_per_second"),
        ("train_steps_per_second", "train_steps_per_second"),
    ]:
        if raw in df.columns and std not in ren:
            ren[raw] = std

    if ren:
        df = df.rename(columns=ren)

    keep = [c for c in df.columns if c.startswith("train_")] + ["epoch"]
    df = df[keep].copy()
    return df.sort_values("epoch")

def collect_metrics_df(root: str, task: str, model: str) -> pd.DataFrame:
    """
    Build a long DataFrame with all epochs for all *_best_accuracy runs of the model.
    Columns:
      task, model, method, rank, run_name, epoch,
      eval_loss, eval_accuracy, eval_runtime, eval_samples_per_second, eval_steps_per_second,
      train_loss, train_runtime, train_samples_per_second, train_steps_per_second
    Missing fields are left as NaN.
    """
    base = Path(root) / task / model
    rows = []

    for method in METHODS:
        mdir = base / method
        if not mdir.exists():
            continue

        for run in sorted(p for p in mdir.iterdir() if p.is_dir() and p.name.endswith("_best_accuracy")):
            rank = parse_rank(run.name) if method != "full_finetuning" else None

            eval_df = read_eval(run / "eval_per_epoch.csv")
            train_df = read_train(run / "train_per_epoch.csv")

            # Merge on epoch (outer to keep whatever exists)
            if eval_df is not None and train_df is not None:
                merged = pd.merge(eval_df, train_df, on="epoch", how="outer", sort=True)
            elif eval_df is not None:
                merged = eval_df.copy()
            elif train_df is not None:
                merged = train_df.copy()
            else:
                # nothing to record for this run
                continue

            merged["task"] = task
            merged["model"] = model
            merged["method"] = method
            merged["rank"] = rank
            merged["run_name"] = run.name
            # Optional: paths for traceability
            merged["eval_csv"] = str(run / "eval_per_epoch.csv") if (run / "eval_per_epoch.csv").exists() else None
            merged["train_csv"] = str(run / "train_per_epoch.csv") if (run / "train_per_epoch.csv").exists() else None

            # Numeric coercion
            for col in ["epoch","eval_loss","eval_accuracy","eval_runtime",
                        "eval_samples_per_second","eval_steps_per_second",
                        "train_loss","train_runtime","train_samples_per_second","train_steps_per_second","rank"]:
                if col in merged.columns:
                    merged[col] = pd.to_numeric(merged[col], errors="coerce")

            rows.append(merged)

    if not rows:
        return pd.DataFrame(columns=[
            "task","model","method","rank","run_name","epoch",
            "eval_loss","eval_accuracy","eval_runtime","eval_samples_per_second","eval_steps_per_second",
            "train_loss","train_runtime","train_samples_per_second","train_steps_per_second",
            "eval_csv","train_csv"
        ])

    df_all = pd.concat(rows, ignore_index=True).sort_values(["method","rank","epoch"], na_position="first")
    # Reorder columns nicely
    front = ["task","model","method","rank","run_name","epoch"]
    eval_cols = [c for c in df_all.columns if c.startswith("eval_")]
    train_cols = [c for c in df_all.columns if c.startswith("train_")]
    path_cols = ["eval_csv","train_csv"] if {"eval_csv","train_csv"} <= set(df_all.columns) else []
    df_all = df_all[front + eval_cols + train_cols + path_cols]
    return df_all

# Build it
df_all = collect_metrics_df(ROOT, TASK, MODEL)
df_all
# Optional: save
# df_all.to_csv(f"{TASK}_{MODEL}_metrics_all_epochs.csv", index=False)


,task,model,method,rank,run_name,epoch,eval_loss,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,eval_csv,train_loss,train_csv,eval_csv,train_csv
0,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,1.0,0.698951,0.476534,0.2235,1239.479,22.373,glue_results_final/rte/roberta-large/full_fine...,0.7147,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
1,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,2.0,0.545070,0.740072,0.2315,1196.702,21.601,glue_results_final/rte/roberta-large/full_fine...,0.6438,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
2,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,3.0,0.420702,0.805054,0.2358,1174.666,21.203,glue_results_final/rte/roberta-large/full_fine...,0.4443,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
3,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,4.0,0.413439,0.833935,0.2260,1225.847,22.127,glue_results_final/rte/roberta-large/full_fine...,0.2840,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
4,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,5.0,0.515210,0.819495,0.2385,1161.564,20.967,glue_results_final/rte/roberta-large/full_fine...,0.1889,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,rte,roberta-large,rslora,1024.0,rslora_r1024_alpha1024_lr0.0001_best_accuracy,1.0,0.691954,0.527076,1.5799,175.323,3.165,glue_results_final/rte/roberta-large/rslora/rs...,0.7561,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
56,rte,roberta-large,rslora,1024.0,rslora_r1024_alpha1024_lr0.0001_best_accuracy,2.0,0.693571,0.472924,1.6385,169.055,3.052,glue_results_final/rte/roberta-large/rslora/rs...,0.7087,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
57,rte,roberta-large,rslora,1024.0,rslora_r1024_alpha1024_lr0.0001_best_accuracy,3.0,0.692899,0.527076,1.5998,173.142,3.125,glue_results_final/rte/roberta-large/rslora/rs...,0.7140,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
58,rte,roberta-large,rslora,1024.0,rslora_r1024_alpha1024_lr0.0001_best_accuracy,4.0,0.707320,0.472924,1.7121,161.794,2.920,glue_results_final/rte/roberta-large/rslora/rs...,0.7076,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...


In [90]:
# === EDA Step 1: summaries from df_all ===

third_epoch = (
    df_all.sort_values("epoch")
          .groupby("run_name", as_index=False)
          .nth(2)
)
# 2) specifically epoch 5 per run (if present)
epoch5 = (
    df_all[df_all["epoch"] == 3]
    .copy()
)

# quick glance
print("Third epoch summary:")
display(third_epoch[["method","rank","epoch","eval_loss","eval_accuracy","train_loss"]].sort_values(["method", "eval_loss"], ascending=[True,True]))

# print("Epoch 5 summary:")
# display(epoch5[["method","rank","epoch","eval_loss","eval_accuracy","train_loss"]].head())


Third epoch summary:


,method,rank,epoch,eval_loss,eval_accuracy,train_loss
2,full_finetuning,NaN,3.0,0.420702,0.805054,0.4443
17,lora,16.0,3.0,0.394365,0.844765,0.4471
27,lora,256.0,3.0,0.396537,0.830325,0.4248
52,lora,8.0,3.0,0.407923,0.848375,0.4717
12,lora,128.0,3.0,0.417348,0.826715,0.4227
42,lora,512.0,3.0,0.433891,0.819495,0.4055
22,lora,1.0,3.0,0.438385,0.812274,0.5236
37,lora,4.0,3.0,0.448264,0.826715,0.4085
47,lora,64.0,3.0,0.487885,0.826715,0.3603
32,lora,32.0,3.0,0.506719,0.765343,0.5982


In [ ]:
# 1) first epoch per run
third_epoch = (
    df_all.sort_values("epoch")
          .groupby("run_name", as_index=False)
          .nth(2)
)

In [87]:
third_epoch

,task,model,method,rank,run_name,epoch,eval_loss,eval_accuracy,eval_runtime,eval_samples_per_second,eval_steps_per_second,eval_csv,train_loss,train_csv,eval_csv,train_csv
8,rte,roberta-large,lora,1024.0,lora_r1024_alpha1024_lr1.77827941e-05_best_acc...,4.0,0.604435,0.646209,13.1558,21.055,0.076,glue_results_final/rte/roberta-large/lora/lora...,0.6451,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...
58,rte,roberta-large,rslora,1024.0,rslora_r1024_alpha1024_lr0.0001_best_accuracy,4.0,0.707320,0.472924,1.7121,161.794,2.920,glue_results_final/rte/roberta-large/rslora/rs...,0.7076,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
18,rte,roberta-large,lora,16.0,lora_r16_alpha16_lr0.00031622776601683794_best...,4.0,0.407530,0.859206,0.4465,620.370,11.198,glue_results_final/rte/roberta-large/lora/lora...,0.2900,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...
98,rte,roberta-large,rslora,64.0,rslora_r64_alpha64_lr1.77827941e-05_best_accuracy,4.0,0.447371,0.844765,0.9770,283.515,5.118,glue_results_final/rte/roberta-large/rslora/rs...,0.3116,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
38,rte,roberta-large,lora,4.0,lora_r4_alpha4_lr0.001_best_accuracy,4.0,0.475128,0.859206,0.4436,624.399,11.271,glue_results_final/rte/roberta-large/lora/lora...,0.2431,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...
63,rte,roberta-large,rslora,128.0,rslora_r128_alpha128_lr1.77827941e-05_best_acc...,4.0,0.388970,0.855596,1.0021,276.424,4.990,glue_results_final/rte/roberta-large/rslora/rs...,0.3582,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
78,rte,roberta-large,rslora,256.0,rslora_r256_alpha256_lr0.00031622776601683794_...,4.0,0.703742,0.472924,0.4684,591.383,10.675,glue_results_final/rte/roberta-large/rslora/rs...,0.7057,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
23,rte,roberta-large,lora,1.0,lora_r1_alpha1_lr0.001_best_accuracy,4.0,0.454148,0.830325,0.4372,633.563,11.436,glue_results_final/rte/roberta-large/lora/lora...,0.3779,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...,glue_results_final/rte/roberta-large/lora/lora...
93,rte,roberta-large,rslora,512.0,rslora_r512_alpha512_lr0.0001_best_accuracy,4.0,0.708001,0.472924,0.5314,521.288,9.410,glue_results_final/rte/roberta-large/rslora/rs...,0.7058,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...,glue_results_final/rte/roberta-large/rslora/rs...
3,rte,roberta-large,full_finetuning,NaN,full_finetuning_lr1e-05_best_accuracy,4.0,0.413439,0.833935,0.2260,1225.847,22.127,glue_results_final/rte/roberta-large/full_fine...,0.2840,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...,glue_results_final/rte/roberta-large/full_fine...
